In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import seaborn as sns
import pandas as pd
import numpy as np 
import scipy.stats as stats
data_df = pd.read_csv('../../datasets/indian-personal-finance-and-spending-habits.csv')
data_df.head()

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Insurance,Groceries,Transport,...,Desired_Savings,Disposable_Income,Potential_Savings_Groceries,Potential_Savings_Transport,Potential_Savings_Eating_Out,Potential_Savings_Entertainment,Potential_Savings_Utilities,Potential_Savings_Healthcare,Potential_Savings_Education,Potential_Savings_Miscellaneous
0,44637.249636,49,0,Self_Employed,Tier_1,13391.174891,0.000000,2206.490129,6658.768341,2636.970696,...,6200.537192,11265.627707,1685.696222,328.895281,465.769172,195.151320,678.292859,67.682471,0.000000,85.735517
1,26858.596592,34,2,Retired,Tier_2,5371.719318,0.000000,869.522617,2818.444460,1543.018778,...,1923.176434,9676.818733,540.306561,119.347139,141.866089,234.131168,286.668408,6.603212,56.306874,97.388606
2,50367.605084,35,1,Student,Tier_3,7555.140763,4612.103386,2201.800050,6313.222081,3221.396403,...,7050.360422,13891.450624,1466.073984,473.549752,410.857129,459.965256,488.383423,7.290892,106.653597,138.542422
3,101455.600247,21,0,Self_Employed,Tier_3,15218.340037,6809.441427,4889.418087,14690.149363,7106.130005,...,16694.965136,31617.953615,1875.932770,762.020789,1241.017448,320.190594,1389.815033,193.502754,0.000000,296.041183
4,24875.283548,52,4,Professional,Tier_2,4975.056710,3112.609398,635.907170,3034.329665,1276.155163,...,1874.099434,6265.700532,788.953124,68.160766,61.712505,187.173750,194.117130,47.294591,67.388120,96.557076


### 검증 할 사항
- 도시 등급에 따른 지출 차이가 있는가
- 도시 등급에 따른 가처분소득 차이가 있는가
- 도시 등급에 따른 소비 습관 차이가 있는가
- 도시 등급에 따른 목표 저축액 차이가 있는가
- 도시 등급에 따른 절약 액 차이가 있는가
- 가처분소득에 따른 목표 저축액에 차이가 있는가
- 소비 습관에 따른 목표 저축액 차이가 있는가 

### 검증 : 가처분소득에 따른 목표 저축액에 차이가 있는가
- 연속형 / 연속형
- 가처분소득과 목표 저축액의 상관계수가 abs(0.5) 이상인지 : 0.90 으로 강한 양의 관계가 있다.
- 가처분소득과 목표 저축액이 5000개 이상인지, 정규분포인지 : 둘다 정규 분포
- 최종 검정 테스트 (p-value 가 0.05 보다 큰지 작은지) : p-value가 0.0 이므로 대립가설 채택

In [4]:
data_df[['Disposable_Income','Desired_Savings']].corr()

,Disposable_Income,Desired_Savings
Disposable_Income,1.000000,0.905816
Desired_Savings,0.905816,1.000000


In [5]:
results = stats.anderson(data_df['Disposable_Income'])
results

AndersonResult(statistic=1414.632216742928, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=10647.367256729376, scale=11740.637288607528)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [6]:
results.significance_level, results.critical_values

(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

In [7]:
results = stats.anderson(data_df['Desired_Savings'])
results

AndersonResult(statistic=2520.8838501328355, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=4982.878415536119, scale=7733.468188192078)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [8]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

In [10]:
stats.pearsonr(data_df['Disposable_Income'].dropna(), data_df['Desired_Savings'].dropna())

PearsonRResult(statistic=0.9058164093314598, pvalue=0.0)

### 검증 : 도시 등급에 따른 지출 차이가 있는가
- 범주형 / 연속형
- 도시 등급에 따른 지출이 5000개 이상인지, 정규분포인지 : 도시 등급에 대한 지출은 정규분포
- 도시 등급의 범주가 3개 이상인지 : 도시 등급인 3개이다.
- 도시 등급에 따른 지출이 등분산인지 : pvalue=0.45 > 0.05 등분산이다.
- 최종 검정 테스트 (p-value 가 0.05 보다 큰지 작은지) : 
  - pvalue=3.17e-189 < 0.05 이므로 대립가설 채택 

In [11]:
data_df['City_Tier'].value_counts()

City_Tier
Tier_2    10068
Tier_1     5934
Tier_3     3998
Name: count, dtype: int64

In [12]:
condition_triple = 'City_Tier in ("Tier_1", "Tier_2", "Tier_3")'
triple_df = data_df.query(condition_triple).dropna()
len(triple_df)

20000

In [13]:
results = stats.anderson(triple_df['Income'].dropna())
results

AndersonResult(statistic=1290.5637343818598, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=41585.49610420809, scale=40014.5405375311)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [14]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

In [15]:
condition_tier1 = 'City_Tier == "Tier_1"'
condition_tier2 = 'City_Tier == "Tier_2"'
condition_tier3 = 'City_Tier == "Tier_3"'
df_tier1 = data_df.query(condition_tier1).dropna()
df_tier2 = data_df.query(condition_tier2).dropna()
df_tier3 = data_df.query(condition_tier3).dropna()

len(df_tier1), len(df_tier2), len(df_tier3)

(5934, 10068, 3998)

In [18]:
test_list = [df_tier1['Income'], df_tier2['Income'], df_tier3['Income']]
stats.levene(*test_list)

LeveneResult(statistic=0.7981966693362551, pvalue=0.4501543250934851)

In [24]:
stats.f_oneway(*test_list)

F_onewayResult(statistic=443.5906781333559, pvalue=3.1782327099561377e-189)

### 검증 : 도시 등급에 따른 가처분소득 차이가 있는가
- 범주형 / 연속형
- 도시 등급에 따른 가처분소득이 5000개 이상인지, 정규분포인지 : 도시 등급에 대한 가처분소득은 정규분포
- 도시 등급의 범주가 3개 이상인지 : 도시 등급인 3개이다.
- 도시 등급에 따른 가처분소득이 등분산인지 : pvalue=6.33e-72 < 0.05 이분산이다.
- 최종 검정 테스트 (p-value 가 0.05 보다 큰지 작은지) : 
  - pvalue=0.0 < 0.05 이므로 대립가설 채택 

In [20]:
results = stats.anderson(triple_df['Disposable_Income'].dropna())
results

AndersonResult(statistic=1414.632216742928, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=10647.367256729376, scale=11740.637288607528)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [21]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

In [22]:
test_list = [df_tier1['Disposable_Income'], df_tier2['Disposable_Income'], df_tier3['Disposable_Income']]
stats.levene(*test_list)

LeveneResult(statistic=165.29156885840797, pvalue=6.334142144017232e-72)

In [23]:
stats.kruskal(*test_list)


KruskalResult(statistic=1592.172752685743, pvalue=0.0)